In [26]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np 


In [27]:
## Load the ANN trained model, scaler pickle file and Onehot encoder pickle file
model = load_model('model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

## Important: Feature Names Must Match Training Data

The scaler and model were trained on specific feature names. Make sure your input data has the **exact same column names** as the training data:
- Use `IsActiveMember` (not `ArithmeticErrors`)
- Column order should match the training data order

In [34]:
## Example input data for prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',   
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,  # Fixed: Changed from 'ArithmeticErrors' to 'IsActiveMember'
    'EstimatedSalary': 50000
}

In [35]:
## Changing the categorical data to numerical using the loaded encoders
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\PROJECTS\Deep Learning\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [36]:
## Conver the input data into dataframe
input_df = pd.DataFrame([input_data])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [37]:
## Encode all my categorical data to numerical data
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df 


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [38]:
## Concatenate the encoded geography columns with the input dataframe and drop the original Geography column
input_df = pd.concat([input_df,geo_encoded_df],axis=1)
input_df.drop(['Geography'],axis=1,inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [39]:
### Scaling the input Data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [40]:
## Now Predict the output
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 21s 21s/step


array([[0.0397881]], dtype=float32)

In [41]:
prediction_proba = prediction[0][0]


In [42]:
prediction_proba

0.0397881

In [44]:
if prediction_proba > 0.5:
    print("Customer will likely to churn (EXIT)")
else:
    print("Customer is not likely to churn (STAY)")

Customer is not likely to churn (STAY)
